In [1]:
import gymnasium as gym
import ale_py
from gymnasium.wrappers.resize_observation import ResizeObservation
from gymnasium.wrappers.gray_scale_observation import GrayScaleObservation
from gymnasium.wrappers.frame_stack import FrameStack
from gymnasium.wrappers.transform_observation import TransformObservation

from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticCnnPolicy
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env.vec_frame_stack import VecFrameStack

def make_env():
    env = gym.make(
        'ALE/MsPacman-v5',
        obs_type='grayscale',
        render_mode="rgb_array",
        frameskip=5,
        repeat_action_probability=0.25,
        mode=0
    )
    env = ResizeObservation(env=env, shape=64)
    # env = GrayScaleObservation(env=env, keep_dim=False)
    # env = FrameStack(env, num_stack=4)
    # env = TransformObservation(env, lambda x: x.transpose(1,2,0,3))
    env = Monitor(env)
    return env

env = make_env()

# env = DummyVecEnv([make_env for _ in range(1)])
# env = VecFrameStack(env, n_stack=4)


env.reset()[0].shape

A.L.E: Arcade Learning Environment (version 0.10.1+unknown)
[Powered by Stella]


(64, 64)

In [3]:
from stable_baselines3.common.atari_wrappers import AtariWrapper



def make_env(n_envs: int = 1):
    def _make_env():
        env = gym.make(
            'ALE/MsPacman-v5',
            obs_type='rgb',
            render_mode="rgb_array",
            mode=0
        )
        env = AtariWrapper(
            env,
            noop_max=250,
            frame_skip=5,
            screen_size=64,
            terminal_on_life_loss=True,
            clip_reward=True,
            action_repeat_probability=0.0,
            grayscale_obs=False,
        )
        env = Monitor(env)

        return env

    env = DummyVecEnv([_make_env for _ in range(n_envs)])
    env = VecFrameStack(env, n_stack=4)

    return env

env = make_env(n_envs=2)


env.reset().shape

(2, 64, 64, 12)

In [18]:
import gymnasium as gym
import ale_py
from gymnasium.wrappers.resize_observation import ResizeObservation
from gymnasium.wrappers.frame_stack import FrameStack

from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticCnnPolicy
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env.vec_video_recorder import VecVideoRecorder

   
gym.register_envs(ale_py)

def make_env():
    env = gym.make(
        'ALE/MsPacman-v5',
        obs_type='rgb',
        render_mode="rgb_array",
        frameskip=5,
        repeat_action_probability=0.25,
        mode=0
    )
    env = ResizeObservation(env=env, shape=64)
    # env = GrayScaleObservation(env=env, keep_dim=False)
    env = FrameStack(env, num_stack=4)
    env = Monitor(env)
    return env

# List of hyperparameters to try
n_envs = 1
n_steps = 128

env = DummyVecEnv([make_env for _ in range(n_envs)])
mini_batch_size = n_steps  # Mini-batch size often equals n_steps for simplicity

# Creating the PPO model
ppo = PPO(
    policy=ActorCriticCnnPolicy,
    env=env,
    learning_rate=lambda _: 1e-4,  # Constant learning rate
    n_steps=n_steps,
    batch_size=mini_batch_size,
    n_epochs=4,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.1,
    clip_range_vf=1,
    normalize_advantage=True,
    ent_coef=0.001,
    vf_coef=1,
    target_kl=None,
    tensorboard_log='logs',
    device='cuda',
    verbose=1
)

Using cuda device


AssertionError: You should use NatureCNN only with images not with Box(0, 255, (4, 64, 64, 3), uint8)
(you are probably using `CnnPolicy` instead of `MlpPolicy` or `MultiInputPolicy`)
If you are using a custom environment,
please check it using our env checker:
https://stable-baselines3.readthedocs.io/en/master/common/env_checker.html.
If you are using `VecNormalize` or already normalized channel-first images you should pass `normalize_images=False`: 
https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html